# Subida de datos locales

In [1]:
import pandas as pd

ruta = r"C:\Users\camil\Downloads\Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.csv"

df = pd.read_csv(ruta)

print(df.head())
print(df.shape)

            Timestamp How old are you?  What industry do you work in?  \
0  4/27/2021 11:02:10            25-34   Education (Higher Education)   
1  4/27/2021 11:02:22            25-34              Computing or Tech   
2  4/27/2021 11:02:38            25-34  Accounting, Banking & Finance   
3  4/27/2021 11:02:41            25-34                     Nonprofits   
4  4/27/2021 11:02:42            25-34  Accounting, Banking & Finance   

                                  Job title  \
0        Research and Instruction Librarian   
1  Change & Internal Communications Manager   
2                      Marketing Specialist   
3                           Program Manager   
4                        Accounting Manager   

  If your job title needs additional context, please clarify here:  \
0                                                NaN                 
1                                                NaN                 
2                                                NaN                

### Exploracion de datos

In [4]:
from ydata_profiling import ProfileReport

# Generar perfil
profile = ProfileReport(
    df,
    title="Ask A Manager Salary Survey 2021 - Data Profiling",
    explorative=True
)

# Exportar a HTML
profile.to_file("salary_survey_profiling.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 18/18 [00:08<00:00,  2.09it/s][A


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Reasignacion de normbres

In [3]:
df.columns

Index(['Timestamp', 'How old are you?', 'What industry do you work in?',
       'Job title',
       'If your job title needs additional context, please clarify here:',
       'What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)',
       'How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.',
       'Please indicate the currency',
       'If "Other," please indicate the currency here: ',
       'If your income needs additional context, please provide it here:',
       'What country do you work in?',
       'If you're in the U.S., what state do you work in?',
       'What city do you work in?',
       'How many years of professional work experience do you have overall?',
       

In [5]:
df = df.rename(columns={
    'Timestamp': 'timestamp',
    'How old are you?': 'age',
    'What industry do you work in?': 'industry',
    'Job title': 'job_title',
    'If your job title needs additional context, please clarify here:': 'job_title_context',
    'What is your annual salary? (You\'ll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)': 'annual_salary_raw',
    'How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.': 'additional_compensation_raw',
    'Please indicate the currency': 'currency',
    'If "Other," please indicate the currency here: ': 'currency_other',
    'If your income needs additional context, please provide it here:': 'income_context',
    'What country do you work in?': 'country_raw',
    'If you\'re in the U.S., what state do you work in?': 'state',
    'What city do you work in?': 'city_raw',
    'How many years of professional work experience do you have overall?': 'years_experience_total',
    'How many years of professional work experience do you have in your field?': 'years_experience_field',
    'What is your highest level of education completed?': 'education',
    'What is your gender?': 'gender',
    'What is your race? (Choose all that apply.)': 'race'
})

In [6]:
text_cols = [
    'country_raw', 'city_raw', 'industry',
    'currency', 'currency_other'
]

for col in text_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .str.lower()
        .replace({'nan': None, '': None})
    )

In [8]:
df['country'] = df['country_raw']
df['city'] = df['city_raw']

In [9]:
df['country_clean'] = (
    df['country']
    .astype(str)
    .str.lower()
    .str.strip()
)

In [10]:
df['city_clean'] = (
    df['city']
    .astype(str)
    .str.lower()
    .str.strip()
)

# Normalizacion de paises y ciudades

In [14]:
usa_values = {
    "u.s.", "united states", "us", "america", "the united states",
    "the us", "u. s", "u. s.", "u.a.", "u.s", "u.s.",
    "u.s>", "u.sa", "ua", "u.s.a.", "usa", "u.s.a",
    "uxz", "united y", "united statss", "uniteed states",
    "usa (company is based in a us territory, i work remote)",
    "usa-- virgin islands", "usa, but for foreign gov't",
    "uniited states", "unite states", "united states of america",
    "unites states", "usa tomorrow", "unitef stated",
    "united states of american", "united stares", "united state",
    "united state of america", "united stated", "united stateds",
    "united states is america", "unitedstates"
}

uk_values = {
    "united kingdom", "uk", "england", "scotland", "wales"
}

In [16]:
def normalize_country(x):
    if x in usa_values:
        return "USA"
    elif x in uk_values:
        return "UK"
    else:
        return x

df['country_normalized'] = df['country_clean'].apply(normalize_country)

In [18]:
df[['country', 'country_normalized']].value_counts().head(20)

country                   country_normalized
united states             USA                   10052
usa                       USA                    9082
us                        USA                    2787
canada                    canada                 1682
uk                        UK                      693
united kingdom            UK                      634
u.s.                      USA                     608
united states of america  USA                     492
australia                 australia               390
germany                   germany                 198
england                   UK                      170
ireland                   ireland                 125
new zealand               new zealand             123
france                    france                   69
netherlands               netherlands              57
spain                     spain                    50
u.s.a.                    USA                      48
scotland                  UK         

In [19]:
import unicodedata
import re

def clean_text(x):
    if pd.isna(x):
        return None
    x = x.lower().strip()
    x = unicodedata.normalize('NFKD', x)
    x = x.encode('ascii', 'ignore').decode('utf-8')
    x = re.sub(r'[^a-z\s]', '', x)
    x = re.sub(r'\s+', ' ', x)
    return x

In [20]:
def print_city_matches(df, patterns, column='city_clean', top=None):
    mask = df[column].str.contains('|'.join(patterns), na=False)
    result = (
        df.loc[mask, column]
        .value_counts()
        .reset_index()
        .rename(columns={'index': column, column: 'count'})
    )
    if top:
        result = result.head(top)
    return result


In [21]:
city_freq = (
    df['city_clean']
    .value_counts()
    .reset_index()
    .rename(columns={'index': 'city_clean', 'city_clean': 'count'})
)

city_freq.head(30)

,count,count
0,chicago,834
1,boston,832
2,new york,820
3,seattle,743
4,london,626
5,san francisco,590
6,los angeles,540
7,new york city,533
8,portland,487
9,toronto,444


In [22]:
patterns_new = ['^new ', 'new york', 'nyc']
print_city_matches(df, patterns_new)

,count,count
0,new york,820
1,new york city,533
2,nyc,236
3,new orleans,65
4,new haven,29
...,...,...
70,nyc (remotely),1
71,"nyc, currently wfh",1
72,new delhi,1
73,new westminster,1


In [23]:
patterns_wash = ['^wash', 'washington']
print_city_matches(df, patterns_wash)

,count,count
0,washington,386
1,"washington, dc",260
2,washington dc,169
3,"washington, d.c.",34
4,washington d.c.,15
5,"washington, d.c",2
6,eastern washington,2
7,fort washington,2
8,port washington,2
9,tiverton (remote) but company based in washington,1


In [24]:
# San / Santa
print_city_matches(df, ['^san ', '^santa '])

# Los / Las
print_city_matches(df, ['^los ', '^las '])

# Port
print_city_matches(df, ['^port'])

# Remote / híbridos
print_city_matches(df, ['remote', 'home', 'hybrid'])


,count,count
0,remote,161
1,work from home,7
2,home,5
3,remote worker,4
4,from home,3
...,...,...
173,"remote work in ca, company is in wa",1
174,boulder but we are remote,1
175,i work remotely from a different location.,1
176,home plus travel to customers. near chicago,1


In [25]:
new_york_values = {
    "new york",
    "new york city",
    "nyc",
    "new york ny",
    "new york new york",
    "new york city new york",
    "nyc ny",
    "new city ny",
    "new york citybrooklyn",
    "new york citymanhattan",
    "new york city the bronx",
    "nyc queens"
}
washington_dc_values = {
    "washington dc",
    "washington",
    "washingtondc",
    "washington district of columbia",
    "washington dc area",
    "washington dc metro area",
    "washington dc suburbs",
    "washington dc primarily"
}
remote_values = {
    "remote",
    "work from home",
    "home",
    "remote worker",
    "from home",
    "remotely",
    "fully remote",
    "home based",
    "remote based",
    "remote job",
    "remote work",
    "currently remote"
}


In [27]:
def normalize_city(x):
    if x is None:
        return None

    # Remote primero (regla dura)
    if (
        "remote" in x
        or "wfh" in x
        or "work from home" in x
        or x in remote_values
    ):
        return "Remote"

    # New York
    elif x in new_york_values:
        return "New York"

    # Washington DC
    elif x in washington_dc_values:
        return "Washington DC"

    else:
        return x.title()


In [32]:
df['city_normalized'] = df['city_clean'].apply(normalize_city)

In [35]:
df[['city_clean', 'city_normalized']].value_counts().head(30)

city_clean      city_normalized
chicago         Chicago            834
boston          Boston             832
new york        New York           820
seattle         Seattle            743
london          London             626
san francisco   San Francisco      590
los angeles     Los Angeles        540
new york city   New York           533
portland        Portland           487
toronto         Toronto            444
minneapolis     Minneapolis        389
washington      Washington DC      386
atlanta         Atlanta            362
austin          Austin             356
philadelphia    Philadelphia       351
denver          Denver             335
houston         Houston            279
washington, dc  Washington, Dc     260
dallas          Dallas             242
pittsburgh      Pittsburgh         241
vancouver       Vancouver          240
nyc             New York           236
cambridge       Cambridge          210
columbus        Columbus           196
madison         Madison         

In [36]:
df['city_normalized'].value_counts().head(20)

city_normalized
New York          1589
Chicago            834
Boston             832
Seattle            743
London             626
San Francisco      590
Washington DC      558
Los Angeles        540
Portland           487
Toronto            444
Minneapolis        389
Remote             367
Atlanta            362
Austin             356
Philadelphia       351
Denver             335
Houston            279
Washington, Dc     260
Dallas             242
Pittsburgh         241
Name: count, dtype: int64

# Salario

In [44]:
df['annual_salary_raw'].sample(15, random_state=1)

16766     75,000
13957    109,270
2407     156,000
26054      76500
7373      68,000
17523     88,000
12542     58,500
22190     110000
3914      72,000
3773      51,000
5751     104,000
4036      63,550
19679     41,600
7574      65,000
24028      58000
Name: annual_salary_raw, dtype: object

In [45]:
df['currency'].unique()

array(['usd', 'gbp', 'cad', 'eur', 'aud/nzd', 'other', 'chf', 'zar',
       'sek', 'hkd', 'jpy'], dtype=object)

In [46]:
df['annual_salary_raw'].isna().mean()

0.0

In [47]:
(
    df['annual_salary_raw']
        .dropna()
        .astype(str)
        .str.contains('[a-zA-Z]')
        .value_counts()
)


annual_salary_raw
False    28190
Name: count, dtype: int64

In [48]:
(
    df['annual_salary_raw']
        .dropna()
        .astype(str)
        .str.contains('[$kK]')
        .value_counts()
)

annual_salary_raw
False    28190
Name: count, dtype: int64

In [49]:
df['annual_salary'] = (
    df['annual_salary_raw']
        .astype(str)
        .str.replace(',', '', regex=False)
)

df['annual_salary'] = pd.to_numeric(df['annual_salary'], errors='coerce')

In [50]:
allowed_currencies = ["usd", "gbp", "eur", "chf", "cad", "aud/nzd"]

mask = (
    df["currency"].str.lower().isin(allowed_currencies)
    & (df["annual_salary"] < 100)
)

df.loc[mask, "annual_salary"] = df.loc[mask, "annual_salary"] * 1000

In [51]:
import pandas as pd

fx_rates = pd.DataFrame({
    "currency": [
        "usd", "gbp", "cad", "eur", "aud/nzd",
        "other", "chf", "zar", "sek", "hkd", "jpy"
    ],
    "cop_rate": [
        3699.04, 5034.82, 2708.84, 4374.50, 2583.41,
        0.0, 4767.32, 231.39, 408.35, 473.34, 23.58
    ]
})

In [52]:
df = df.merge(
    fx_rates,
    on="currency",
    how="left"
)

# --- 3. Conversión a COP ---
df['annual_salary_cop'] = (
    df['annual_salary'] * df['cop_rate']
)

# --- 4. Flags de control (obligatorio) ---
df['fx_error'] = df['cop_rate'].isna() | (df['cop_rate'] == 0)

In [53]:
df[df['fx_error']][['currency', 'annual_salary']]

,currency,annual_salary
434,other,885000
603,other,1080000
1311,other,80640
1840,other,60000
1924,other,800000
...,...,...
28119,other,10000
28144,other,19200
28150,other,288000
28184,other,72000


In [54]:
df['annual_salary_cop'].describe(percentiles=[.01, .05, .5, .95, .99])

count    2.819000e+04
mean     9.228544e+08
std      9.683521e+10
min      0.000000e+00
1%       4.060630e+07
5%       1.202955e+08
50%      2.737290e+08
95%      6.658272e+08
99%      9.851746e+08
max      1.625323e+13
Name: annual_salary_cop, dtype: float64

In [56]:
df["additional_compensation_raw_cop"] = (
    df["additional_compensation_raw"] * df["cop_rate"]
)

In [57]:
df[
    df["additional_compensation_raw"].notna()
][[
    "currency",
    "additional_compensation_raw",
    "cop_rate",
    "additional_compensation_raw_cop"
]].head(20)


,currency,additional_compensation_raw,cop_rate,additional_compensation_raw_cop
0,usd,0.0,3699.04,0.0
1,gbp,4000.0,5034.82,20139280.0
3,usd,3000.0,3699.04,11097120.0
4,usd,7000.0,3699.04,25893280.0
6,usd,2000.0,3699.04,7398080.0
8,usd,10000.0,3699.04,36990400.0
9,usd,0.0,3699.04,0.0
10,usd,0.0,3699.04,0.0
11,usd,0.0,3699.04,0.0
12,usd,0.0,3699.04,0.0


In [58]:
df["salario_total_cop"] = (
    df["annual_salary_cop"].fillna(0)
    + df["additional_compensation_raw_cop"].fillna(0)
)


In [59]:
df[[
    "annual_salary_cop",
    "additional_compensation_raw_cop",
    "salario_total_cop"
]].head(20)


,annual_salary_cop,additional_compensation_raw_cop,salario_total_cop
0,203447200.0,0.0,203447200.0
1,274901172.0,20139280.0,295040452.0
2,125767360.0,NaN,125767360.0
3,229340480.0,11097120.0,240437600.0
4,221942400.0,25893280.0,247835680.0
5,229340480.0,NaN,229340480.0
6,122068320.0,7398080.0,129466400.0
7,184952000.0,NaN,184952000.0
8,414292480.0,36990400.0,451282880.0
9,166456800.0,0.0,166456800.0


In [60]:
(df["salario_total_cop"] < df["annual_salary_cop"]).sum()


0

In [117]:
df['years_experience_field'].unique()

array(['5-7 years', '2 - 4 years', '21 - 30 years', '11 - 20 years',
       '1 year or less', '8 - 10 years', '31 - 40 years',
       '41 years or more'], dtype=object)

In [119]:
df['years_experience_total'].unique()


array(['5-7 years', '8 - 10 years', '2 - 4 years', '21 - 30 years',
       '11 - 20 years', '1 year or less', '41 years or more',
       '31 - 40 years'], dtype=object)

In [121]:
experience_map = {
    '1 year or less': 0.5,
    '2 - 4 years': 3,
    '5-7 years': 6,
    '8 - 10 years': 9,
    '11 - 20 years': 15.5,
    '21 - 30 years': 25.5,
    '31 - 40 years': 35.5,
    '41 years or more': 45
}

df['years_experience_field_mean'] = df['years_experience_field'].map(experience_map)
df['years_experience_total_mean'] = df['years_experience_total'].map(experience_map)

In [123]:
df.to_csv(
    "df_salarios_procesado.csv",
    index=False,
    encoding="utf-8"
)

In [62]:
from ydata_profiling import ProfileReport

# Generar perfil
profile = ProfileReport(
    df,
    title="Ask A Manager Salary Survey 2021 - Data Profiling - 2",
    explorative=True
)

# Exportar a HTML
profile.to_file("salary_survey_profiling_2.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 30/30 [00:09<00:00,  3.12it/s][A


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [63]:
df.columns

Index(['timestamp', 'age', 'industry', 'job_title', 'job_title_context',
       'annual_salary_raw', 'additional_compensation_raw', 'currency',
       'currency_other', 'income_context', 'country_raw', 'state', 'city_raw',
       'years_experience_total', 'years_experience_field', 'education',
       'gender', 'race', 'country', 'city', 'country_clean', 'city_clean',
       'country_normalized', 'city_normalized', 'annual_salary', 'cop_rate',
       'annual_salary_cop', 'fx_error', 'additional_compensation_raw_cop',
       'salario_total_cop'],
      dtype='object')

In [113]:
df_filtrado = df[
    (df["currency"] == "chf") &
    (df["country_normalized"].str.lower() == "india")
]


In [115]:
df_filtrado.head()

,timestamp,age,industry,job_title,job_title_context,annual_salary_raw,additional_compensation_raw,currency,currency_other,income_context,...,country_clean,city_clean,country_normalized,city_normalized,annual_salary,cop_rate,annual_salary_cop,fx_error,additional_compensation_raw_cop,salario_total_cop
21293,4/30/2021 1:16:12,25-34,ipr,Sr. Research associate,NaN,1000000,NaN,chf,rs,NaN,...,india,delhi,india,Delhi,1000000,4767.32,4.767320e+09,False,NaN,4.767320e+09
